In [1]:
import pandas as pd
import numpy as np
import os
import hashlib
from clean_payroll_script import get_cleaned_df
from sqlalchemy import create_engine
import pymysql
import getpass  # To get the password without showing the input
import mysql.connector
import time
from sqlalchemy import text
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
os.getcwd()

'c:\\Users\\bfran\\Ironhack\\New-York-Payroll-Analysis'

In [12]:
# Set pandas display options to avoid scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# Ensure payroll information is saved in current working directory as 'Payroll.csv'
# Pulls function from clean_payroll_script to get and clean payroll data and save as csv for future use
# payroll = get_cleaned_df()

c:\Users\bfran\Ironhack\New-York-Payroll-Analysis\clean_payroll_script.py:11: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  payroll = pd.read_csv('Payroll.csv')


In [3]:
payroll = pd.read_csv('cleaned_payroll.csv')

In [4]:
payroll

,worker_id,fiscal_year,agency_name,agency_start_date,work_location_borough,title_description,leave_status_as_of_june_30,base_salary,pay_basis,regular_hours,regular_gross_paid,ot_hours,total_ot_paid,total_other_pay,converted_salary,total_pay,true_index
0,MBEPM_08102015,2020,OFFICE OF EMERGENCY MANAGEMENT,2015-08-10,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00,84698.21,0
1,VGEPM_09122016,2020,OFFICE OF EMERGENCY MANAGEMENT,2016-09-12,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00,84698.21,1
2,SREPM_02222016,2020,OFFICE OF EMERGENCY MANAGEMENT,2016-02-22,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00,84698.21,2
3,JREPM_09162013,2020,OFFICE OF EMERGENCY MANAGEMENT,2013-09-16,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00,84698.21,3
4,RWEPM_04302018,2020,OFFICE OF EMERGENCY MANAGEMENT,2018-04-30,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00,84698.21,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662083,JZP_10022017,2023,NYC HOUSING AUTHORITY,2017-10-02,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1149.00,77575.47,3752.01,315.07,163245.68,5662083
5662084,LZP_08212017,2023,NYC HOUSING AUTHORITY,2017-08-21,BRONX,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1037.75,70063.81,3206.96,315.07,155188.97,5662084
5662085,PZSP_10072019,2023,NYC HOUSING AUTHORITY,2019-10-07,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,PER DAY,1820.0,93620.80,908.00,70035.56,617.28,360.08,164273.64,5662085
5662086,HZP_07062022,2023,NYC HOUSING AUTHORITY,2022-07-06,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1771.0,79082.62,1005.75,67869.89,0.00,315.07,146952.51,5662086


In [5]:
# Save password object and connect python to MySQL
password = getpass.getpass()
conn = mysql.connector.connect(host='localhost',
                        user='root',
                        passwd=password)

In [6]:
pr = 'payroll'
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+pr
engine = create_engine(connection_string)     # saves the connection string as object engine

In [7]:
conn = mysql.connector.connect(host='localhost',
                        user='root',
                        passwd=password)

cursor = conn.cursor()

In [16]:
# Function to read last processed index from a file
def read_last_processed_index(filename):
    try:
        with open(filename, 'r') as file:
            last_index = int(file.read().strip())
    except FileNotFoundError:
        last_index = 0
    return last_index

# Function to write last processed index to a file
def write_last_processed_index(filename, index):
    with open(filename, 'w') as file:
        file.write(str(index))

# File to store last processed index
last_processed_file = 'last_processed_index.txt'

# Read last processed index from file
last_processed_index = read_last_processed_index(last_processed_file)

# Read last processed index from file
last_processed_index = read_last_processed_index(last_processed_file)

chunksize = 10000
num_chunks = len(payroll) // chunksize + 1

try:
    # Track start time for total elapsed time
    start_time_total = time.time()

    # Read the CSV file in chunks and process/upload to MySQL
    for i, chunk in enumerate(pd.read_csv('cleaned_payroll.csv', chunksize=chunksize, skiprows=range(0, last_processed_index))):
        start_time_chunk = time.time()  # Start time for current chunk

        with engine.begin() as connection:
            chunk.to_sql(name='payroll', con=connection, if_exists='append', index=False)

        # Calculate elapsed time for current chunk
        elapsed_time_chunk = time.time() - start_time_chunk
        print(f"Chunk {i+1 + last_processed_index}/{num_chunks} uploaded in {elapsed_time_chunk:.2f} seconds.")

        # Update last processed index in file
        write_last_processed_index(last_processed_file, i * chunksize + len(chunk))

except Exception as e:
    print(f"Error: {str(e)}")
    # Handle the error appropriately (retry, log, etc.)

finally:
    # Calculate total elapsed time
    end_time_total = time.time()
    elapsed_time_total = end_time_total - start_time_total
    print(f"Total elapsed time: {elapsed_time_total:.2f} seconds")

    # Close the SQLAlchemy engine
    engine.dispose()

Chunk 1/567 uploaded in 2.35 seconds.
Chunk 2/567 uploaded in 0.78 seconds.
Chunk 3/567 uploaded in 0.65 seconds.
Chunk 4/567 uploaded in 0.59 seconds.
Chunk 5/567 uploaded in 0.61 seconds.
Chunk 6/567 uploaded in 0.57 seconds.
Chunk 7/567 uploaded in 0.60 seconds.
Chunk 8/567 uploaded in 0.81 seconds.
Chunk 9/567 uploaded in 0.55 seconds.
Chunk 10/567 uploaded in 0.60 seconds.
Chunk 11/567 uploaded in 0.58 seconds.
Chunk 12/567 uploaded in 0.55 seconds.
Chunk 13/567 uploaded in 0.56 seconds.
Chunk 14/567 uploaded in 0.53 seconds.
Chunk 15/567 uploaded in 0.53 seconds.
Chunk 16/567 uploaded in 0.55 seconds.
Chunk 17/567 uploaded in 0.54 seconds.
Chunk 18/567 uploaded in 0.57 seconds.
Chunk 19/567 uploaded in 0.54 seconds.
Chunk 20/567 uploaded in 0.55 seconds.
Chunk 21/567 uploaded in 0.55 seconds.
Chunk 22/567 uploaded in 0.53 seconds.
Chunk 23/567 uploaded in 0.57 seconds.
Chunk 24/567 uploaded in 0.62 seconds.
Chunk 25/567 uploaded in 0.55 seconds.
Chunk 26/567 uploaded in 0.52 seco

In [91]:
with engine.connect() as connection:
    # Getting how many loans were granted every year, and the month of each duration.
    txt = '''SELECT ot.agency_name, 
	                ot.fiscal_year, 
                    ac.agency_count, 
                    ot.agency_ot/1920 as staff_needed,
                    LEAD(ac.agency_count, 1) OVER (PARTITION BY ac.agency_name ORDER BY ac.fiscal_year) - ac.agency_count AS staff_hired,
                    (LEAD(ac.agency_count, 1) OVER (PARTITION BY ac.agency_name ORDER BY ac.fiscal_year) - ac.agency_count) - (ot.agency_ot/1920) as difference
            FROM ot_hours as ot
            JOIN agency_changes as ac
            ON ot.agency_name = ac.agency_name AND ot.fiscal_year = ac.fiscal_year
            WHERE ot.agency_ot/1920 > 1 
            ORDER BY ABS(difference) DESC;'''
    query = text(txt)
    result = connection.execute(query)
    needvgot = pd.DataFrame(result.all())

In [76]:
needvgot

,agency_name,fiscal_year,agency_count,staff_needed,staff_hired,difference
0,POLICE DEPARTMENT,2022,60804,8634.736052,-2187.0,-10821.736052
1,POLICE DEPARTMENT,2020,60316,9257.166073,-1307.0,-10564.166073
2,POLICE DEPARTMENT,2016,65868,6442.557953,-3352.0,-9794.557953
3,DEPT OF PARKS & RECREATION,2015,24535,234.327990,-9249.0,-9483.327990
4,DEPT OF PARKS & RECREATION,2014,15120,222.348417,9415.0,9192.651583
...,...,...,...,...,...,...
489,RICHMOND DA,2014,112,1.416536,NaN,None
490,TAX COMMISSION,2022,70,1.420703,NaN,None
491,TAXI & LIMOUSINE COMMISSION,2023,672,10.175391,NaN,None
492,TEACHERS RETIREMENT SYSTEM,2022,439,1.856318,NaN,None


In [92]:
# clean up a bit
def replace_none(dif, fy):
    if dif == None and fy == 2023:
        return 'Not Reported'
    elif dif == None and fy != 2023:
        return 0
    else:
        return dif  # Return the original value if no replacement is needed

# Apply the function using apply method
needvgot['difference'] = needvgot.apply(lambda row: replace_none(row['difference'], row['fiscal_year']), axis=1)

columns = ['staff_needed','staff_hired']
needvgot[columns].fillna(0, inplace=True)
needvgot[columns].replace('None',0,inplace=True)
# Ensure all values are numeric by converting the entire column to float before rounding and casting
for column in columns:
    needvgot[column] = pd.to_numeric(needvgot[column], errors='coerce').fillna(0)
needvgot[columns] = needvgot[columns].round().astype(int)
needvgot

C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\523722020.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needvgot[columns].fillna(0, inplace=True)
C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\523722020.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needvgot[columns].replace('None',0,inplace=True)


,agency_name,fiscal_year,agency_count,staff_needed,staff_hired,difference
0,POLICE DEPARTMENT,2022,60804,8635,-2187,-10821.736052
1,POLICE DEPARTMENT,2020,60316,9257,-1307,-10564.166073
2,POLICE DEPARTMENT,2016,65868,6443,-3352,-9794.557953
3,DEPT OF PARKS & RECREATION,2015,24535,234,-9249,-9483.327990
4,DEPT OF PARKS & RECREATION,2014,15120,222,9415,9192.651583
...,...,...,...,...,...,...
489,RICHMOND DA,2014,112,1,0,0
490,TAX COMMISSION,2022,70,1,0,0
491,TAXI & LIMOUSINE COMMISSION,2023,672,10,0,Not Reported
492,TEACHERS RETIREMENT SYSTEM,2022,439,2,0,0


In [100]:
# Remove Not Reporteds
needvgot = needvgot[~(needvgot['difference']=='Not Reported')]

# Subset data
needvgot_pos = needvgot[needvgot['difference'] > 0]
needvgot_neg = needvgot[(needvgot['difference'] <= 0) & (needvgot['staff_needed'] > needvgot['agency_count'] / 10)]

In [103]:
# Display df where the agency with the greatest negative difference between staff needed and hired is listed first, w/ each year's rows being ordered chronologically.
# Step 1: Calculate the absolute maximum difference per agency
max_abs_diff_per_agency = needvgot_neg.groupby('agency_name')['difference'].apply(lambda x: np.abs(x).max())
needvgot_neg['max_abs_difference'] = needvgot_neg['agency_name'].map(max_abs_diff_per_agency)

# Step 2: Sort the DataFrame
needvgot_neg= needvgot_neg.sort_values(by=['max_abs_difference', 'agency_name', 'fiscal_year'],
                                       ascending=[False, True, True])

# Step 3: Drop the temporary column if needed
needvgot_neg.drop('max_abs_difference', axis=1, inplace=True)

needvgot_neg

C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\4023994085.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needvgot_neg['max_abs_difference'] = needvgot_neg['agency_name'].map(max_abs_diff_per_agency)


,agency_name,fiscal_year,agency_count,staff_needed,staff_hired,difference
42,POLICE DEPARTMENT,2014,55619,5586,3645,-1940.644307
190,POLICE DEPARTMENT,2015,59264,6746,6604,-141.548948
7,POLICE DEPARTMENT,2018,59811,6076,159,-5917.416380
8,POLICE DEPARTMENT,2019,59970,6040,346,-5694.353193
1,POLICE DEPARTMENT,2020,60316,9257,-1307,-10564.166073
0,POLICE DEPARTMENT,2022,60804,8635,-2187,-10821.736052
37,FIRE DEPARTMENT,2014,16555,2995,825,-2170.300839
97,FIRE DEPARTMENT,2015,17380,3040,2554,-485.972932
11,FIRE DEPARTMENT,2016,19934,2960,-1247,-4207.492755
21,FIRE DEPARTMENT,2017,18687,2879,-296,-3174.814057


In [104]:
# Display df where the agency with the greatest positive difference between staff needed and hired is listed first, w/ each year's rows being ordered chronologically.
# Step 1: Calculate the absolute maximum difference per agency
max_abs_diff_per_agency = needvgot_pos.groupby('agency_name')['difference'].apply(lambda x: np.abs(x).max())
needvgot_pos['max_abs_difference'] = needvgot_pos['agency_name'].map(max_abs_diff_per_agency)

# Step 2: Sort the DataFrame
needvgot_pos = needvgot_pos.sort_values(by=['max_abs_difference', 'agency_name', 'fiscal_year'],
                                       ascending=[False, True, True])

# Step 3: Drop the temporary column if needed
needvgot_pos.drop('max_abs_difference', axis=1, inplace=True)

needvgot_pos

C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\3225939547.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needvgot_pos['max_abs_difference'] = needvgot_pos['agency_name'].map(max_abs_diff_per_agency)


,agency_name,fiscal_year,agency_count,staff_needed,staff_hired,difference
4,DEPT OF PARKS & RECREATION,2014,15120,222,9415,9192.651583
185,DEPT OF PARKS & RECREATION,2016,15286,270,416,146.274052
61,DEPT OF PARKS & RECREATION,2018,15574,300,1413,1113.285667
43,DEPT OF PARKS & RECREATION,2021,10414,274,2190,1915.689297
15,DEPT OF PARKS & RECREATION,2022,12604,341,4128,3787.116125
...,...,...,...,...,...,...
396,BUSINESS INTEGRITY COMMISSION,2021,91,2,7,5.227474
379,BUSINESS INTEGRITY COMMISSION,2022,98,1,10,8.626172
381,DEPARTMENT OF BUSINESS SERV.,2017,389,1,9,7.944271
402,OFFICE OF THE ACTUARY,2014,36,1,5,3.816797


In [124]:
# Get agency names into a list
neg_deps = needvgot_neg['agency_name'].drop_duplicates()

# Iteration loop that gets and prints summary statistics for each agency
for i in neg_deps:
    agency_data = needvgot_neg[needvgot_neg['agency_name'] == i]
    agency_data.drop(columns='fiscal_year',inplace=True)
    summary_stats = agency_data.describe().loc[['count', 'min', 'mean', '50%', 'max']].T
    
    print(f"Summary statistics for {i}:")
    print(summary_stats)
    print("\n")

Summary statistics for POLICE DEPARTMENT:
              count      min          mean      50%      max
agency_count    6.0  55619.0  59297.333333  59890.5  60804.0
staff_needed    6.0   5586.0   7056.666667   6411.0   9257.0
staff_hired     6.0  -2187.0   1210.000000    252.5   6604.0


Summary statistics for FIRE DEPARTMENT:
              count      min          mean      50%      max
agency_count    9.0  16555.0  18754.222222  18687.0  20920.0
staff_needed    9.0   2749.0   3441.888889   2995.0   5188.0
staff_hired     9.0  -1848.0    279.666667    288.0   2554.0


Summary statistics for DEPARTMENT OF CORRECTION:
              count      min       mean      50%      max
agency_count    8.0  11135.0  13141.375  13345.5  14292.0
staff_needed    8.0   1258.0   2045.750   1941.5   2796.0
staff_hired     8.0  -1920.0   -402.625   -179.5    388.0


Summary statistics for DEPARTMENT OF SANITATION:
              count      min          mean      50%      max
agency_count    6.0  11787.0  123

C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\2503730477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agency_data.drop(columns='fiscal_year',inplace=True)
C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\2503730477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agency_data.drop(columns='fiscal_year',inplace=True)
C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\2503730477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [126]:
# Get agency names into a list
pos_deps = needvgot_pos['agency_name'].drop_duplicates()

# Iteration loop that gets and prints summary statistics for each agency
for i in pos_deps:
    agency_data = needvgot_pos[needvgot_pos['agency_name'] == i]
    agency_data.drop(columns='fiscal_year',inplace=True)
    summary_stats = agency_data.describe().loc[['count', 'min', 'mean', '50%', 'max']].T
    
    print(f"Summary statistics for {i}:")
    print(summary_stats)
    print("\n")

C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\715936937.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agency_data.drop(columns='fiscal_year',inplace=True)
C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\715936937.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agency_data.drop(columns='fiscal_year',inplace=True)
C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\715936937.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

Summary statistics for DEPT OF PARKS & RECREATION:
              count      min     mean      50%      max
agency_count    5.0  10414.0  13799.6  15120.0  15574.0
staff_needed    5.0    222.0    281.4    274.0    341.0
staff_hired     5.0    416.0   3512.4   2190.0   9415.0


Summary statistics for COMMUNITY COLLEGE (MANHATTAN):
              count     min    mean     50%     max
agency_count    5.0  4622.0  5229.8  5147.0  5813.0
staff_needed    5.0    28.0    32.4    32.0    37.0
staff_hired     5.0   194.0  1040.8   326.0  3759.0


Summary statistics for COMMUNITY COLLEGE (LAGUARDIA):
              count     min    mean     50%     max
agency_count    5.0  3813.0  4015.6  3977.0  4266.0
staff_needed    5.0    10.0    12.8    14.0    14.0
staff_hired     5.0    92.0   840.0   289.0  2971.0


Summary statistics for COMMUNITY COLLEGE (KINGSBORO):
              count     min         mean     50%     max
agency_count    3.0  2625.0  3540.333333  3669.0  4327.0
staff_needed    3.0     3.0

C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\715936937.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agency_data.drop(columns='fiscal_year',inplace=True)
C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\715936937.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agency_data.drop(columns='fiscal_year',inplace=True)
C:\Users\bfran\AppData\Local\Temp\ipykernel_13420\715936937.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [2]:
ranking = pd.read_excel('ranking_w_boroughs.xlsx')

In [5]:
ranking.columns

Index(['Rank', 'Geographic Area', 'April 1, 2020\nEstimate Base',
       'July 1, 2020 Estimate', 'July 1, 2021 Estimate',
       'July 1, 2022 Estimate', 'July 1, 2023 Estimate'],
      dtype='object')

In [7]:
# forward fill nans with the percent change in population of the row above
percent_changes = ranking.copy()

# Get year to year percent changes
for col1, col2 in zip(['April 1, 2020\nEstimate Base', 'July 1, 2020 Estimate', 'July 1, 2021 Estimate', 'July 1, 2022 Estimate'],
                      ['July 1, 2020 Estimate', 'July 1, 2021 Estimate','July 1, 2022 Estimate', 'July 1, 2023 Estimate']):
    percent_changes[col2] = (ranking[col2] - ranking[col1]) / ranking[col1]

# Forward fill NaNs using the percent change from the previous row
for i in range(1, len(ranking)):
    for col1, col2 in zip(['April 1, 2020\nEstimate Base', 'July 1, 2020 Estimate', 'July 1, 2021 Estimate', 'July 1, 2022 Estimate'],
                          ['July 1, 2020 Estimate', 'July 1, 2021 Estimate','July 1, 2022 Estimate', 'July 1, 2023 Estimate']):
        if pd.isnull(ranking.at[i, col2]):
            ranking.at[i, col2] = ranking.at[i, col1] * (1 + percent_changes.at[i-1, col2])

In [13]:
ranking

,Rank,Geographic Area,"April 1, 2020\nEstimate Base","July 1, 2020 Estimate","July 1, 2021 Estimate","July 1, 2022 Estimate","July 1, 2023 Estimate"
0,1,"Los Angeles city, California",3898841,3895848,3832573.00,3822782.00,3820914.00
1,2,"Chicago city, Illinois",2746352,2743329,2704101.00,2672660.00,2664452.00
2,3,"Brooklyn borough, New York city, New York",2538934,2538934,2502628.73,2473530.28,2465933.82
3,4,"Houston city, Texas",2300833,2299269,2291020.00,2302488.00,2314157.00
4,5,"Queens borough, New York city, New York",2225821,2225821,2217835.51,2228937.17,2240233.42
...,...,...,...,...,...,...,...
1908,1909,"Plainview city, Texas",20183,20140,19897.00,19600.00,19420.00
1909,1910,"Harvey city, Illinois",20330,20251,19874.00,19545.00,19397.00
1910,1911,"Ypsilanti city, Michigan",20655,20605,19185.00,19739.00,19393.00
1911,1912,"Albany city, California",20271,20225,19814.00,19349.00,19097.00


In [14]:
cols2round = ['July 1, 2021 Estimate','July 1, 2022 Estimate', 'July 1, 2023 Estimate']
ranking[cols2round] = ranking[cols2round].round()
ranking

,Rank,Geographic Area,"April 1, 2020\nEstimate Base","July 1, 2020 Estimate","July 1, 2021 Estimate","July 1, 2022 Estimate","July 1, 2023 Estimate"
0,1,"Los Angeles city, California",3898841,3895848,3832573.00,3822782.00,3820914.00
1,2,"Chicago city, Illinois",2746352,2743329,2704101.00,2672660.00,2664452.00
2,3,"Brooklyn borough, New York city, New York",2538934,2538934,2502629.00,2473530.00,2465934.00
3,4,"Houston city, Texas",2300833,2299269,2291020.00,2302488.00,2314157.00
4,5,"Queens borough, New York city, New York",2225821,2225821,2217836.00,2228937.00,2240233.00
...,...,...,...,...,...,...,...
1908,1909,"Plainview city, Texas",20183,20140,19897.00,19600.00,19420.00
1909,1910,"Harvey city, Illinois",20330,20251,19874.00,19545.00,19397.00
1910,1911,"Ypsilanti city, Michigan",20655,20605,19185.00,19739.00,19393.00
1911,1912,"Albany city, California",20271,20225,19814.00,19349.00,19097.00


In [15]:
ranking.to_csv('ranking_w_boroughs.csv',index=False)

In [16]:
city_ranks = pd.read_excel('city_pop_ranks.xlsx')

In [17]:
city_ranks.to_csv('city_ranks',index = False)